# SMOTE NN With Genetic Algorithm

In [1]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from tpot import TPOTClassifier, TPOTRegressor
from deap.gp import Primitive
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from ipywidgets import IntProgress
from sklearn import metrics
from sklearn.metrics import make_scorer

In [2]:
Data = pd.read_csv('Dataset/clean/KC2.csv')

In [3]:
X = Data.loc[:, Data.columns != 'class']
y = Data['class']

In [25]:
pipeline = Pipeline([('smt', smt), ('clf', XGBClassifier)])

params = {'clfmax_depth': np.arange(1,200,1),
          'learning_rate': np.arange(0.0001,0.1,0.0001),
          'n_estimators': np.arange(1,200,1),
          'nthread':[6],
          'gamma':np.arange(0.00001,0.1,0.00001),
          'subsample':np.arange(0.1,2,0.1),
          'reg_lambda': np.arange(0.1,200,1),
          'reg_alpha': np.arange(1,200,1),
          'min_child_weight': np.arange(1,200,1),
          'gamma': np.arange(0.1,2,0.1),
          'colsample_bytree': np.arange(0.1,2,0.1),
          'colsample_bylevel': np.arange(0.1,2,0.1)
         }

In [21]:
def gmeans(y_true, y_pred):
    # repro()
    tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    pf = fp/(fp+tn)
    pd = tp/(tp+fn)
    res = math.sqrt(pd*(1-pf))
    return res

In [24]:
gmeans_score = make_scorer(gmeans)

tpot_classifier = TPOTClassifier(generations=20, population_size=10, offspring_size=10,
                                verbosity=2, early_stop=8,
                                config_dict={'': params}, cv = 2, scoring = gmeans_score)
tpot_classifier.fit(X, y)

AttributeError: 'Pipeline' object has no attribute 'split'

In [ ]:
args = {}
for arg in tpot_classifier._optimized_pipeline:
    if type(arg) != Primitive:
        try:
            if arg.value.split('__')[1].split('=')[0] in ['max_depth', 'n_estimators', 'nthread','min_child_weigh']:
                args[arg.value.split('__')[1].split('=')[0]] = int(arg.value.split('__')[1].split('=')[1])
            else:
                args[arg.value.split('__')[1].split('=')[0]] = float(arg.value.split('__')[1].split('=')[1])
        except:
            pass
params = args
params

In [ ]:

xgb = XGBClassifier(**params)

In [27]:
from pyeasyga import pyeasyga
data = [('pear', 50), ('apple', 35), ('banana', 40)]
ga = pyeasyga.GeneticAlgorithm(data)

def fitness (individual, data):
    fitness = 0
    if individual.count(1) == 2:
        for (selected, (fruit, profit)) in zip(individual, data):
            if selected:
                fitness += profit
    return fitness

ga.fitness_function = fitness
ga.run()
print (ga.best_individual())

(90, [1, 0, 1])
